In [1]:
# Load Packages
from __future__ import unicode_literals, print_function

import plac #  wrapper over argparse
import random
from pathlib import Path
import spacy
from tqdm import tqdm, tqdm_notebook # loading bar 
import pandas as pd
import re
from pprint import pprint
from nltk.tokenize import sent_tokenize
import numpy as np
import unicodedata
from spacy.util import minibatch, compounding
import ast as ast
import pickle

In [2]:
# Função para substituir acentos

def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError): # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    
    return str(text)

# Abrindo e gerando Datasets

### Entidade logradouro

In [3]:
with open ('../Datasets/Dataset_V2.0.txt', 'rb') as fp:
    dataset = pickle.load(fp)

print(len(dataset))

FileNotFoundError: [Errno 2] No such file or directory: '../Datasets/Dataset_V2.0.txt'

In [17]:
dataset[0]

('R.GUILHERME MOREIRA,315 SUBLOJA,LOJA E 2.ANDAR 69005-300 MANAUS CENTRO AM',
 {'entities': [(0, 23, 'END_LOG')]})

In [8]:
end_lista = []

for i in range(len(dataset)):
    str_tratada = dataset[i][0]
    str_tratada = str_tratada.lower() # Tudo em minúsculo
    str_tratada = strip_accents(str_tratada) # Substituição de acentos e caracteres diferentes
    
    end_lista.append(str_tratada)

In [32]:
end_lista[:3]

['r.guilherme moreira,315 subloja,loja e 2.andar 69005-300 manaus centro am',
 '69005-300 manaus am r.guilherme moreira,315 centro subloja,loja e 2.andar',
 'r.guilherme moreira,315 centro 69005-300 subloja,loja e 2.andar manaus am']

In [24]:
iob = []

for i in range(len(dataset)):
    tupla = (end_lista[i], dataset[i][1])
    
    iob.append(tupla)
    
FULL_DATA = iob[:5000]

In [25]:
print(FULL_DATA[0])
print(FULL_DATA[1])
print(FULL_DATA[2])

('r.guilherme moreira,315 subloja,loja e 2.andar 69005-300 manaus centro am', {'entities': [(0, 23, 'END_LOG')]})
('69005-300 manaus am r.guilherme moreira,315 centro subloja,loja e 2.andar', {'entities': [(20, 43, 'END_LOG')]})
('r.guilherme moreira,315 centro 69005-300 subloja,loja e 2.andar manaus am', {'entities': [(0, 23, 'END_LOG')]})


In [26]:
# Criação da base de teste e treinamento

n_test= 0.1 # Porcentagem para base de teste
test_n = round(len(FULL_DATA) * n_test)

# Divisao em Train Test Val

def gerador_bases(dataset, n):
    indices_random = random.sample(range(0,len(dataset)-1), n)
    base_teste_n = []
    base_treinamento_n = []
    
    for i in range(n):
        base_teste_n.append(dataset[indices_random[i]])

    for j in range(len(dataset)):
        if(j not in indices_random):
            base_treinamento_n.append(dataset[j])
            
    return base_teste_n, base_treinamento_n


base_teste, base_treinamento = gerador_bases(FULL_DATA, test_n)

random.shuffle(base_treinamento)
random.shuffle(base_teste)

print("Treinamento: " + str(len(base_treinamento)), "\nTeste: " + str(len(base_teste)), "\nTotal: " + str(len(FULL_DATA)))

Treinamento: 4500 
Teste: 500 
Total: 5000


In [27]:
for i in range(5):
    print(base_treinamento[i])

('r.inocencio braga,337 centro terreo 62500-000 itapipoca ce', {'entities': [(0, 21, 'END_LOG')]})
('centro sc concordia 89700-000  r.do comercio,63', {'entities': [(31, 47, 'END_LOG')]})
('ma centro  65520-000 brejo r.dr.silva martins,210', {'entities': [(27, 49, 'END_LOG')]})
('av.antonio piranga,143 diadema 09911-160 sp centro', {'entities': [(0, 22, 'END_LOG')]})
('88320-000 centro sc predio da igreja catolica r.frei jacinto,66 ilhota', {'entities': [(46, 63, 'END_LOG')]})


# Carregando o modelo

In [28]:
# Define our variables

model = None
output_dir=Path(".")
n_iter= 50 # número de épocas
batch_size = 64

In [29]:
# Setting up the pipeline and entity recognizer.
if model is not None:
    nlp = spacy.load(model)  # load existing spacy model
    print("Loaded model '%s'" % model)
else:
    lan = 'pt'
    nlp = spacy.blank(lan)  # create blank Language class
    print("Created blank '%s' model" % lan)
    
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
    print('Added new NER')
else:
    ner = nlp.get_pipe('ner')
    print('Got an old NER')

Created blank 'pt' model
Added new NER


In [30]:
# create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe('ner')

## Treinamento

In [31]:
print("Batch size: ", batch_size)
print("Épocas: ", n_iter)
print()

# add labels
for _, annotations in base_treinamento:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])


# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(base_treinamento)
        losses = {}
        batches = minibatch(base_treinamento, size=batch_size)
        
        for batch in batches:
            texts, annotations = zip(*batch)
            try:
                nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                       losses=losses)
            except:
                pass     
        
        print(itn+1, '\tLosses', losses)        

Batch size:  64
Épocas:  50

1 	Losses {'ner': 6921.200579205528}
2 	Losses {'ner': 424.8661566598703}
3 	Losses {'ner': 40.50866456229545}
4 	Losses {'ner': 31.025566451117943}
5 	Losses {'ner': 18.300163503947005}
6 	Losses {'ner': 5.2368560810716325}
7 	Losses {'ner': 2.378700758060999}
8 	Losses {'ner': 5.499071679788858}
9 	Losses {'ner': 8.618326534592017}
10 	Losses {'ner': 9.253795861803882}
11 	Losses {'ner': 3.5298624201578277}
12 	Losses {'ner': 9.741561972618866}
13 	Losses {'ner': 6.557690606955678}
14 	Losses {'ner': 22.286835792579943}
15 	Losses {'ner': 15.253152650485603}
16 	Losses {'ner': 5.827690352376961}
17 	Losses {'ner': 8.86547559832059}
18 	Losses {'ner': 30.018459907513513}
19 	Losses {'ner': 47.51594375557084}
20 	Losses {'ner': 6.053718036754316}
21 	Losses {'ner': 7.844149821333003}
22 	Losses {'ner': 4.422824001742839}
23 	Losses {'ner': 6.499669204886844}
24 	Losses {'ner': 6.0042208988209165}
25 	Losses {'ner': 13.667927245084657}
26 	Losses {'ner': 8.3

## Teste

Base de Teste

In [32]:
# test the trained model
for text, _ in base_teste:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print(text)
    print()

Entities [('av.visc.de guarapuava,1930', 'END_LOG')]
80060-060 centro  av.visc.de guarapuava,1930 curitiba pr

Entities [('r.quinze de novembro,70', 'END_LOG')]
valinhos centro  r.quinze de novembro,70 sp 13270-130

Entities [('av.barao do rio branco,348', 'END_LOG')]
cacador sc av.barao do rio branco,348  centro 89500-000

Entities [('r.jorge lacerda,1168', 'END_LOG')]
r.jorge lacerda,1168 sao jose do cedro sc centro  89930-000

Entities [('av.quatro de julho,211', 'END_LOG')]
64260-000  centro pi piripiri av.quatro de julho,211

Entities [('r.treze de maio,3083', 'END_LOG')]
terreo cameta pa centro r.treze de maio,3083 68400-000

Entities [('av.cardeal eugenio pacelli,1966', 'END_LOG')]
mg 32210-003 contagem cid.industrial  av.cardeal eugenio pacelli,1966

Entities [('av.firmino maia,46', 'END_LOG')]
av.firmino maia,46 al centro  57700-000 vicosa

Entities [('r.paranaiba,598', 'END_LOG')]
ms centro r.paranaiba,598 tres lagoas 79601-050 predio

Entities [('r.odilon negrao,629', 'END_L

Entities [('r.cap.cardoso de melo,80', 'END_LOG')]
r.cap.cardoso de melo,80  sp 18150-000 ibiuna centro

Entities [('r.amaral costa,72', 'END_LOG')]
campo grande rj r.amaral costa,72 rio de janeiro 23050-260

Entities [('r.dr.monteiro,582', 'END_LOG')]
centro r.dr.monteiro,582 arroio grande 96330-000 rs

Entities [('av.sete de setembro,1183', 'END_LOG')]
97560-000 av.sete de setembro,1183 rs comercial quarai centro

Entities [('pca.cons.mata,23', 'END_LOG')]
centro pca.cons.mata,23 diamantina  39100-000 mg

Entities [('r.dr.alberto torres,4', 'END_LOG')]
centro  sao fidelis rj r.dr.alberto torres,4 28400-000

Entities [('av.da amizade,s/n', 'END_LOG')]
av.da amizade,s/n centro  tabatinga 69640-000 am

Entities [('pca.pref.joao freire carvalho,349', 'END_LOG')]
cabrobo centro 56180-000 pe  pca.pref.joao freire carvalho,349

Entities [('pca.dr.sergio magalhaes,718', 'END_LOG')]
pca.dr.sergio magalhaes,718 centro vizinho ao cactus hotel 56903-910 pe serra talhada

Entities [('r.exped.abil

Entities [('pca.dr.emilio imbassahy,306', 'END_LOG')]
45900-000 terreo ba caravelas centro pca.dr.emilio imbassahy,306

Entities [('r marechal floriano, 85', 'END_LOG')]
centro r marechal floriano, 85  rs 95700-110 bento goncalves

Entities [('r.dos andradas,525', 'END_LOG')]
r.dos andradas,525 predio rs 97573-565 centro santana do livramento

Entities [('av.sete de setembro,55', 'END_LOG')]
se  49680-000 nossa senhora da gloria centro av.sete de setembro,55

Entities [('pca.pref.joao freire carvalho,349', 'END_LOG')]
pca.pref.joao freire carvalho,349 centro  cabrobo 56180-000 pe

Entities [('av.goias,980', 'END_LOG')]
go av.goias,980 centro 3.andar 74010-010 goiania

Entities [('r.caetano furquim,170', 'END_LOG')]
centro vassouras r.caetano furquim,170 rj  27700-000

Entities [('pca.des.pontes vieira,s/n', 'END_LOG')]
centro pca.des.pontes vieira,s/n  61940-165 ce maranguape

Entities [('pca.frei felix,195', 'END_LOG')]
mg 39200-000 corinto centro pca.frei felix,195

Entities [('r.ban

Entities [('r.prof.salatiel almeida,18', 'END_LOG')]
r.prof.salatiel almeida,18 mg centro  37890-000 muzambinho

Entities [('r.coelho neto,35', 'END_LOG')]
centro terreo r.coelho neto,35 sc 89160-000 rio do sul

Entities [('r.bom pastor,169', 'END_LOG')]
r.bom pastor,169 04203-050 terreo ipiranga sp sao paulo

Entities [('r.herval,313', 'END_LOG')]
03062-000 r.herval,313 belenzinho sp  sao paulo

Entities [('pca.pe.oliveira,890', 'END_LOG')]
se pca.pe.oliveira,890 49800-000 centro  porto da folha

Entities [('r.olavo bilac,298', 'END_LOG')]
r.olavo bilac,298 am centro tefe 69470-000 em frente a pca.tulio azevedo

Entities [('r.rodrigo silva,26', 'END_LOG')]
centro  rj r.rodrigo silva,26 20011-040 rio de janeiro

Entities [('r.dos pinheiros,1492', 'END_LOG')]
pinheiros sao paulo sp r.dos pinheiros,1492  05422-002

Entities [('lgo.conego alcindino,27', 'END_LOG')]
centro historico paranagua pr 83203-330  lgo.conego alcindino,27

Entities [('av.cardeal eugenio pacelli,1966', 'END_LOG')]
a

Entities [('qsa 02,lotes 09 a 11,lojas 01 a 07', 'END_LOG')]
brasilia (taguatinga) df qsa 02,lotes 09 a 11,lojas 01 a 07 72015-020 t sul (taguatinga)

Entities [('r.vinte e um de abril,140', 'END_LOG')]
centro sp  16400-030 r.vinte e um de abril,140 lins

Entities [('r.wladislau garcia gomes,1200', 'END_LOG')]
r.wladislau garcia gomes,1200 centro paranaiba 79500-000  ms

Entities [('pca.gen.osorio,133', 'END_LOG')]
centro 36700-000 leopoldina mg pca.gen.osorio,133

Entities [('r.demerval amaral,35', 'END_LOG')]
centro r.demerval amaral,35 es 29560-000 guacui terreo

Entities [('pca.dr.fernando abbott,23', 'END_LOG')]
pca.dr.fernando abbott,23 97300-000 sao gabriel rs centro

Entities [('pca.jose bonifacio,945', 'END_LOG')]
pca.jose bonifacio,945 piracicaba sp centro 13400-340

Entities [('pca.dr.henrique sampaio,150', 'END_LOG')]
45745-000 pca.dr.henrique sampaio,150 ibicarai ba centro

Entities [('av.tupi,2581', 'END_LOG')]
85501-268 centro av.tupi,2581 pr pato branco

Entities [('av.

Frase qualquer

In [49]:
# Função para tratamento de texto de entrada

def regex(str_):
    ph_tratada = str_.replace("\"", "")
    ph_tratada = ph_tratada.replace("-", " ")
    ph_tratada = ph_tratada.replace("|", "")
#     ph_tratada = ph_tratada.replace(",", "")
    ph_tratada = ph_tratada.replace(":", "")
    ph_tratada = ph_tratada.replace(".", "")
#     ph_tratada = ph_tratada.replace("  ", "")
    ph_tratada = ph_tratada.lower()
    ph_tratada = re.sub(r'[\n]+', " ", ph_tratada)
    ph_tratada = re.sub(r'^[ ]+', "", ph_tratada)

    ph_tratada = strip_accents(ph_tratada
                              )    
    return ph_tratada

In [67]:
ph = "SHVP Rua 12 435 casa 22 lote 1"
ph1 = "SHIN QL 6 conjunto 2 casa 16 Lago Norte"
ph2 = "SQSW 102 bloco D ap 406"
ph3 = "QNE 20,12 TAGUATINGA NORTE 72125-200 TAGUATINGA-DF"

In [80]:
phrase = ph3

regex_ph = regex(phrase)
doc = nlp(regex_ph)
# doc = nlp(phrase)

print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
print(regex_ph)
# print(phrase)

Entities []
qne 20,12 taguatinga norte 72125 200 taguatinga df


In [37]:
texto = [
        "",
        "",
        "PREFEITURA DE Certidão de Dados Cadastrais do Imóvel - IPTU 2017\n\nSÃO PAULO\n\nFAZENDA",
        "Cadastro do Imóvel: 010.012.0164-7",
        "| Local do Imóvel:",
        "| R AUGUSTA,476- A2 523",
        "CEP 01304-000\nImóvel localizado na 12 Subdivisão da Zona Urbana",
        "| Endereço para entrega da notificação:",
        "| R AUGUSTA,476- A2 523",
        "| CEP 01304-000",
        "| Contribuinte(s):",
        "| CNP) 16.942.935/0001-27  WGROW NEGOCIOS DE BENS PROPRIOS E PARTICIPACOE",
        "| Dados cadastrais do terreno:",
        "| Área incorporada (m?): 416 Testada (m): 13,00",
        "| Área não incorporada (m?): 0 Fração ideal: 0,0435",
        "| Área total (m?):\n\n416",
        "| Dados cadastrais da construção:",
        "| Área construída (m?):\n\n99\n\nPadrão da construção:\n\n2-C",
        "Área ocupada pela construção (m?): 380 Uso: residência",
        "| Ano da construção corrigido: 1970",
        "| Valores de m? (R$):",
        "| - de terreno: 5.226,00",
        "| - da construção: 2.007,00",
        "| Valores para fins de cálculo do IPTU (R$):",
        "| - da área incorporada: 151.312,00",
        "- da área não incorporada: 0,00",
        "| - da construção: 87.425,00",
        "| Base de cálculo do IPTU: 238.737,00",
        "Ressalvado o direito da Fazenda Pública do Município de São Paulo atualizar os dados constantes do Cadastro\nImobiliário Fiscal, apurados ou verificados a qualquer tempo, inclusive em relação ao exercício abrangido por\nesta certidão, a Secretaria Municipal da Fazenda CERTIFICA que os dados cadastrais acima foram utilizados no\nlançamento do Imposto Predial e Territorial Urbano do imóvel do exercício de 2017.",
        "Certidão expedida via Internet - Portaria SF nº 008/2004, de 28/01/2004.",
        "A autenticidade desta certidão poderá ser confirmada, até o dia 12/03/2018, em",
        "http://www .prefeitura.sp.gov.br/cidade/secretarias/financas/servicos/certidoes/",
        "| Data de Emissão: 12/12/2017",
        "| Número do Documento: 2.2017.001772627-7",
        "Solicitante: WGROW NEGOCIOS DE BENS PROPRIOS E PARTICIPACOES LTDA. (CNP)\n16.942.935/0001-27)"
    
]

In [50]:
texto
for each in texto:
    texto_tratado = regex(each)
    doc = nlp(texto_tratado)
    
    print(texto_tratado)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print()


Entities []


Entities []

prefeitura de certidao de dados cadastrais do imovel   iptu 2017 sao paulo fazenda
Entities []

cadastro do imovel 0100120164 7
Entities []

local do imovel
Entities []

r augusta,476  a2 523
Entities [('r augusta,476', 'END_LOG')]

cep 01304 000 imovel localizado na 12 subdivisao da zona urbana
Entities []

endereco para entrega da notificacao
Entities []

r augusta,476  a2 523
Entities [('r augusta,476', 'END_LOG')]

cep 01304 000
Entities []

contribuinte(s)
Entities []

cnp) 16942935/0001 27  wgrow negocios de bens proprios e participacoe
Entities [('16942935/0001 27', 'END_LOG')]

dados cadastrais do terreno
Entities []

area incorporada (m?) 416 testada (m) 13,00
Entities []

area nao incorporada (m?) 0 fracao ideal 0,0435
Entities []

area total (m?) 416
Entities []

dados cadastrais da construcao
Entities []

area construida (m?) 99 padrao da construcao 2 c
Entities []

area ocupada pela construcao (m?) 380 uso residencia
Entities []

ano da construc